In [1]:
import csv

skip_head = True
rows = []
with open("online_retail.csv") as f:
    for cols in csv.reader(f):
        if skip_head:
            skip_head = False
            labels = cols
            continue
        if len(cols) == 8 and cols[0][0] != "C":
            rows.append(cols)

In [2]:
from collections import defaultdict

invoices = defaultdict(lambda: set())
for row in rows:
    invoices[row[0]].add(row[2])

In [4]:
all_items_set = set()
for items in invoices.values():
    all_items_set.update(items)
len(all_items_set)
all_items = sorted(list(all_items_set))

In [5]:
from tqdm import tqdm

presence_matrix = []
for invoice in tqdm(invoices.values()):
    row = [ int(item in invoice) for item in all_items ]
    presence_matrix.append(row)

100%|██████████| 22064/22064 [00:40<00:00, 546.95it/s]


In [6]:
def get_presence_matrix(invoices, all_items):
    item_pos_dict = { k: v for v, k in enumerate(all_items) }
    presence_matrix = []
    for invoice in invoices.values():
        row = [0] * len(all_items)
        for item in invoice:
            row[item_pos_dict[item]] = 1
        presence_matrix.append(row)
    return presence_matrix

presence_matrix = get_presence_matrix(invoices, all_items)

In [8]:
import pandas as pd

df = pd.DataFrame(data=presence_matrix, columns=all_items)

In [12]:
a = df.columns.values

In [13]:
len(a)

4208

In [15]:
b = df.head(20)

In [17]:
c = df.shape
c

(22064, 4208)

In [9]:
from mlxtend.frequent_patterns import fpgrowth

for minsup in tqdm([ 0.5, 0.1, 0.05, 0.02, 0.01 ]):
    freq_itemsets = fpgrowth(df, minsup)
    print(f"{minsup} => {len(freq_itemsets)}")

 20%|██        | 1/5 [00:04<00:17,  4.41s/it]

0.5 => 0


 40%|████      | 2/5 [00:11<00:18,  6.11s/it]

0.1 => 1


 60%|██████    | 3/5 [00:20<00:14,  7.31s/it]

0.05 => 23


 80%|████████  | 4/5 [00:37<00:11, 11.03s/it]

0.02 => 303


100%|██████████| 5/5 [01:08<00:00, 13.66s/it]

0.01 => 1472
